In [4]:
import cv2
import numpy as np
import os
from sklearn.utils import shuffle

def softplus(x):
    """Softplus activation function."""
    return np.log(1 + np.exp(x))

def der_softplus(x):
    """Derivative of the softplus function."""
    return 1 / (1 + np.exp(-x))

def load_data(data_folders):
    """Load images, resize to 128x128, flatten to 49152 vector, and store labels."""
    x_data, t = [], []  # t is the label vector

    for label, folder_path in enumerate(data_folders):
        if not os.path.exists(folder_path):
            print(f"Folder does not exist: {folder_path}")
            continue

        print(f"Loading data from: {folder_path}")
        for filename in os.listdir(folder_path):
            image_path = os.path.join(folder_path, filename)
            try:
                img = cv2.imread(image_path)
                if img is None:
                    raise FileNotFoundError(f"Image could not be read: {image_path}")

                # Resize to 128x128, flatten to 1x49152, and normalize to [0, 1]
                img_resized = cv2.resize(img, (128, 128))
                img_vector = img_resized.flatten().astype(np.float32) / 255.0

                x_data.append(img_vector)
                t.append(label)  # Label 0 for camera, 1 for pizza

            except Exception as e:
                print(f"Error processing image {image_path}: {e}")

    x_data = np.array(x_data)
    # Center all features to zero
    x_data -= np.mean(x_data, axis=0)
    return x_data, np.array(t)

def train_perceptron(inputs, t, weights, rho, iterNo):
    """Train the perceptron using softplus activation and gradient descent."""
    inputs, t = shuffle(inputs, t)  # Shuffle data before training
    for _ in range(iterNo):
        for i in range(len(inputs)):
            x_i = np.insert(inputs[i], 0, 1)  # Add bias term
            output = softplus(np.dot(weights, x_i))  # Forward pass
            error = t[i] - output  # Calculate error

            # Calculate gradient and clip to prevent large updates
            gradient = rho * error * der_softplus(output) * x_i
            gradient = np.clip(gradient, -0.1, 0.1)  # Clip gradient

            weights += gradient
            if np.isnan(weights).any():
                print("NaN detected in weights, check learning rate or data values.")
                return weights
    return weights

def test_perceptron(sample_test, weights):
    """Test the perceptron algorithm on an image vector."""
    sample_test_with_bias = np.insert(sample_test, 0, 1)  # Add bias term
    output = softplus(np.dot(weights, sample_test_with_bias))
    return output

# Paths for training data folders
train_folders = [
    "/content/train/train/camera",  # Label 0
    "/content/train/train/pizza"    # Label 1
]

# Load and prepare training data
x_train, y_train = load_data(train_folders)
print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")

if x_train.shape[0] == 0 or y_train.shape[0] == 0:
    raise Exception("Training data not loaded!")

# Initialize weights with small random values
weights = np.random.randn(x_train.shape[1] + 1) * 0.01  # More balanced weight initialization

# Train the perceptron
rho = 0.0001  # Learning rate
iterNo = 1000  # Number of iterations
weights = train_perceptron(x_train, y_train, weights, rho, iterNo)

# Save the weights for later use
np.save('weights.npy', weights)

# Load weights for testing
weights = np.load('weights.npy')

# Paths for test data folders
test_folders = {
    "camera": "/content/test/test/camera",
    "pizza": "/content/test/test/pizza"
}

total_images, correct_predictions = 0, 0

for category, test_folder_path in test_folders.items():
    print(f"Testing folder: {category}")

    if not os.path.exists(test_folder_path):
        print(f"Test folder does not exist: {test_folder_path}")
        continue

    for filename in os.listdir(test_folder_path):
        test_image_path = os.path.join(test_folder_path, filename)

        try:
            img = cv2.imread(test_image_path)
            if img is None:
                print(f"Image could not be read: {test_image_path}")
                continue

            img_resized = cv2.resize(img, (128, 128))
            test_sample_vector = img_resized.flatten().astype(np.float32) / 255.0

            predicted_output = test_perceptron(test_sample_vector, weights)
            predicted_label = 0 if predicted_output < 0.5 else 1

            expected_label = 0 if category == "camera" else 1
            print(f"{filename} - Expected: {expected_label}, Predicted: {predicted_label}")

            total_images += 1
            if predicted_label == expected_label:
                correct_predictions += 1

        except Exception as test_error:
            print(f"Error in test image {test_image_path}: {test_error}")

# Calculate and display accuracy
if total_images > 0:
    accuracy = (correct_predictions / total_images) * 100
    print(f"Accuracy: %{accuracy:.2f}")
else:
    print("No images were tested.")


Loading data from: /content/train/train/camera
Loading data from: /content/train/train/pizza
x_train shape: (82, 49152), y_train shape: (82,)
Testing folder: camera
image_0024.jpg - Expected: 0, Predicted: 0
image_0040.jpg - Expected: 0, Predicted: 0
image_0037.jpg - Expected: 0, Predicted: 0
image_0046.jpg - Expected: 0, Predicted: 0
image_0035.jpg - Expected: 0, Predicted: 0
Testing folder: pizza
image_0024.jpg - Expected: 1, Predicted: 1
image_0040.jpg - Expected: 1, Predicted: 1
image_0037.jpg - Expected: 1, Predicted: 1
image_0046.jpg - Expected: 1, Predicted: 0
image_0035.jpg - Expected: 1, Predicted: 0
image_0038.jpg - Expected: 1, Predicted: 1
Accuracy: %81.82


In [2]:
from google.colab import files
uploaded = files.upload()


Saving test.zip to test.zip
Saving train.zip to train.zip


In [3]:
!unzip train.zip -d /content/train
!unzip test.zip -d /content/test


Archive:  train.zip
   creating: /content/train/train/
   creating: /content/train/train/camera/
  inflating: /content/train/train/camera/image_0001.jpg  
  inflating: /content/train/train/camera/image_0002.jpg  
  inflating: /content/train/train/camera/image_0003.jpg  
  inflating: /content/train/train/camera/image_0004.jpg  
  inflating: /content/train/train/camera/image_0005.jpg  
  inflating: /content/train/train/camera/image_0006.jpg  
  inflating: /content/train/train/camera/image_0007.jpg  
  inflating: /content/train/train/camera/image_0008.jpg  
  inflating: /content/train/train/camera/image_0009.jpg  
  inflating: /content/train/train/camera/image_0010.jpg  
  inflating: /content/train/train/camera/image_0012.jpg  
  inflating: /content/train/train/camera/image_0013.jpg  
  inflating: /content/train/train/camera/image_0014.jpg  
  inflating: /content/train/train/camera/image_0015.jpg  
  inflating: /content/train/train/camera/image_0016.jpg  
  inflating: /content/train/train